In [1]:
if 'google.colab' in str(get_ipython()):
    import sys
    from google.colab import drive, output
    drive.mount('/content/drive')
    !pip install torchaudio
    !pip install wandb --upgrade
    output.clear()
    print("Running on colab")
    %load_ext autoreload
    %autoreload 1
    %cd '/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project'
    root = "/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project"
else:
    print("Running local")
    root = "/home/mirp_ai/Documents/Daniel_Nieto/PhD/AutoEncoders_Ecoacoustics"
    root_path = "/media/mirp_ai/Seagate Desktop Drive/Jaguas_2018"


Running local


In [2]:
import os
import numpy as np

import datetime
from datetime import timedelta

import torch
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
from torchvision.utils import make_grid
from Jaguas_DataLoader_rainless import SoundscapeData

import pandas as pd

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


cuda


In [3]:
class df_generator():
    def __init__(self, root_clusters, root_audios, root_recorders, n_clusters_index = 0):
        self.n_clusters_index = n_clusters_index
        self.cluster_names = os.listdir(f"{root_clusters}")
        self.torch_clusters = torch.load(f"{root_clusters}/{self.cluster_names[n_clusters_index]}")
        self.dataframe_clusters = pd.DataFrame(self.torch_clusters)
        self.dataframe_clusters = self.dataframe_clusters.transpose()
        self.dataframe_audios = pd.read_csv(f"{root_audios}", index_col=0)
        self.dataframe_recorders = pd.read_csv(f"{root_recorders}",sep = ";", index_col = "Recorder")
        self.df_clusters_len = len(self.dataframe_clusters)
        self.df_recorders_len = len(self.dataframe_recorders)
        self.n_clusters = np.arange(len(self.torch_clusters))

    def show_clusters(self):
        print(self.cluster_names)

    def init_clusters(self):
        for cluster in self.n_clusters:
            self.dataframe_recorders[f"Cluster {cluster}"] = 0
    
    def create_clusters(self, count_cluster=False):
        self.init_clusters()
        for cluster in self.n_clusters:
            for i in range(self.df_clusters_len):  
                if self.dataframe_clusters.iloc[i][cluster] != None:
                    self.dataframe_recorders.loc[self.dataframe_clusters.iloc[i][cluster].split("_")[0], f"Cluster {cluster}"] +=1
                else:
                    pass
        if count_cluster == True:
            self.count_cluster_data()
        else:
            pass
        
        return self.dataframe_recorders
    
    def count_cluster_data(self):
        self.dataframe_recorders["Total_Clustering_Audios"] = 0
        for recorder in range(self.df_recorders_len):
            clusters = [f"Cluster {i}" for i in range(0, len(self.n_clusters))]
            clusters.append("Total_Clustering_Audios")
            self.dataframe_recorders.iloc[recorder, -1] = self.dataframe_recorders[clusters].iloc[recorder].sum()
    
    def GLM_dataframe(self):
        GLM = self.dataframe_recorders.copy()
        columns = [f"Cluster {i}" for i in range(0, len(self.n_clusters))]
        columns.append("Total_Clustering_Audios")
        GLM = GLM[columns]
        for cluster in range(len(columns[0:-1])):
            for recorder in range(self.df_recorders_len):
                GLM.iloc[recorder, cluster] = GLM.iloc[recorder, cluster]/GLM.iloc[recorder, -1]
        return GLM
    
    def regions(self):
        self.dataframe_audios["Region"] = 0
        for audio in range(len(prueba)):   
            self.dataframe_audios.loc[audio,"Region"] = self.dataframe_recorders.loc[self.dataframe_audios.loc[audio, "Recorder"]].loc["Region"]
        return self.dataframe_audios
    
    def save(self, root_save=None):
        if root_save != None:
            os.makedirs(f"{self.root_save}/dataframes", exist_ok=True)
            self.dataframe_recorders.to_csv(f"{self.root_save}/dataframes/dataframe_{self.cluster_names[self.n_clusters_index]}_recorders.csv")
            self.dataframe_recorders.to_csv(f"{self.root_save}/dataframes/dataframe_{self.cluster_names[self.n_clusters_index]}_audios.csv") 
            
        else:
            self.dataframe_recorders.to_csv(f"{self.cluster_names[sel.n_clusters_index]}_recorders.csv")
            self.dataframe_audios.to_csv(f"{self.cluster_names[sel.n_clusters_index]}_audios.csv")            

In [4]:
root_audios = f"{root}/Jaguas/Complementary_Files/Audios_Jaguas/Audios_Jaguas.csv"
root_recorders = f"{root}/Jaguas/Complementary_Files/df_grabadoras_reg.csv"
root_clusters = f"{root}/Jaguas/temporal/clusters"
ecological_integrity = f"{root}/Jaguas/Complementary_Files/Indice_Integridad_Ecologica.xlsx"
df_EI = pd.read_excel(ecological_integrity)

In [13]:
dataframe = df_generator(root_clusters, root_audios, root_recorders, 
                         n_clusters_index=3)
dataframe.create_clusters(count_cluster=True)
GLM = dataframe.GLM_dataframe()
GLM = GLM.reset_index().set_index(np.arange(len(GLM)))
GLM["Mean"] = df_EI["Mean"]
GLM["Sum"] = df_EI["Sum"]
GLM["Max"] = df_EI["Max"]
GLM["Min"] = df_EI["Min"]
GLM

,Recorder,Cluster 0,Cluster 1,Cluster 2,Cluster 3,Cluster 4,Total_Clustering_Audios,Mean,Sum,Max,Min
0,G03,0.154825,0.277193,0.151316,0.320175,0.096491,2280,0.572591,78.444947,0.662041,0.242650
1,G04,0.091411,0.331902,0.112883,0.368712,0.095092,1630,0.587104,82.194536,0.645013,0.388676
2,G06,0.023695,0.414951,0.020874,0.445980,0.094499,3545,0.784025,110.547518,0.896008,0.312327
3,G07,0.035317,0.383333,0.038889,0.433333,0.109127,2520,0.370633,51.147362,0.532298,0.081821
4,G08,0.139669,0.300413,0.151240,0.334298,0.074380,2420,0.587971,80.552073,0.728520,0.384826
5,G09,0.016246,0.427731,0.015126,0.442857,0.098039,3570,0.723915,100.624248,0.790780,0.406501
6,G13,0.115399,0.285343,0.129499,0.339889,0.129870,2695,0.571372,80.563485,0.691484,0.360173
7,G15,0.022409,0.406162,0.030532,0.441457,0.099440,3570,0.519462,72.724744,0.562418,0.176436
8,G17,0.173699,0.261918,0.156986,0.310137,0.097260,3650,0.534994,75.434084,0.793996,0.348641
9,G19,0.112551,0.320648,0.107692,0.353846,0.105263,3705,0.447152,64.389824,0.657790,0.337416


In [ ]:
regiones = dataframe.regions()

In [ ]:
len(GLM.columns)-6

In [ ]:
prueba=dataframe.dataframe_audios
recorder = dataframe.dataframe_recorders
prueba["Region"] = 0
for audio in range(len(prueba)):   
    prueba.loc[audio,"Region"] = recorder.loc[prueba.loc[audio, "Recorder"]].loc["Region"]


# Using SK-Learn

In [ ]:
from sklearn import linear_model
clf = linear_model.PoissonRegressor()
clf.fit(X,y)

In [ ]:
clf.score(X,y)

# Using StatsModels

In [14]:
import statsmodels.api as sm
X= GLM[[f"Cluster {i}" for i in range(len(GLM.columns)-6)]]
X = X.to_numpy()
y = GLM["Mean"]
y = y.to_numpy()
gamma_model = sm.GLM(y, X, family=sm.families.Gamma())
gamma_results = gamma_model.fit()
print(gamma_results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                      y   No. Observations:                   31
Model:                            GLM   Df Residuals:                       26
Model Family:                   Gamma   Df Model:                            4
Link Function:           InversePower   Scale:                        0.060844
Method:                          IRLS   Log-Likelihood:                 20.019
Date:                Tue, 18 Jul 2023   Deviance:                       1.6467
Time:                        11:50:24   Pearson chi2:                     1.58
No. Iterations:                     6   Pseudo R-squ. (CS):            0.09999
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1             1.1011      5.782      0.190      0.8

/home/mirp_ai/anaconda3/envs/DANM/lib/python3.10/site-packages/statsmodels/genmod/generalized_linear_model.py:307: DomainWarning: The InversePower link function does not respect the domain of the Gamma family.
  warnings.warn((f"The {type(family.link).__name__} link function "


In [ ]:
import statsmodels.api as sm
data = sm.datasets.scotland.load()
data.exog = sm.add_constant(data.exog)
gamma_model = sm.GLM(data.endog, data.exog, family=sm.families.Gamma())
gamma_results = gamma_model.fit()
print(gamma_results.summary())

# SK-Learn example

In [ ]:
from sklearn import linear_model
clf = linear_model.PoissonRegressor()
X = [[1, 2], [2, 3], [3, 4], [4, 3]]
y = [12, 17, 22, 21]
clf.fit(X, y)
clf.score(X, y)
